In [4]:
import glob
import os
import pandas as pd

!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-2b8-55b-c4-perplexity
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-2b8-55b-c4-repetitions
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-2b8-55b-c4seeds


!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-4b2-84b-c4-perplexity
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-4b2-84b-c4-repetitions
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-4b2-84b-c4seeds

!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-2b8-55b-oscar-perplexity
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-2b8-55b-oscar-repetitions
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-2b8-55b-oscarseeds

!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-4b2-84b-oscar-perplexity
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-4b2-84b-oscar-repetitions
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-4b2-84b-oscarseeds

TASK_TO_BASELINE = {
    "anli_r1": 1/3,
    "anli_r2": 1/3,
    "anli_r3": 1/3,
    "arc_challenge": 1/4,
    "arc_easy": 1/4,
    "boolq": 1/2,
    "cb": 1/3,
    "copa": 1/2,
    "hellaswag": 1/4,
    "piqa": 1/2,
    "rte": 1/2,
    "sciq": 1/4,
    "storycloze_2016": 1/4,
    "winogrande": 1/2,
    "babi": 0,
}

MODELS = [
    "lm1-2b8-55b-c4-repetitions/2b855b55bc4",
    "lm1-2b8-55b-c4seeds/2b855b55bc4seed1",
    "lm1-2b8-55b-c4seeds/2b855b55bc4seed2",
    "lm1-2b8-55b-c4seeds/2b855b55bc4seed3",
    "lm1-2b8-55b-c4seeds/2b855b55bc4seed4",

    "lm1-2b8-55b-c4-perplexity/perplexity25",
    "lm1-2b8-55b-c4-perplexity/perplexity50",


    "lm1-4b2-84b-c4-repetitions/4b284b84bc4",
    "lm1-4b2-84b-c4seeds/4b284b84bc4seed1",
    "lm1-4b2-84b-c4seeds/4b284b84bc4seed2",
    "lm1-4b2-84b-c4seeds/4b284b84bc4seed3",
    "lm1-4b2-84b-c4seeds/4b284b84bc4seed4",

    "lm1-4b2-84b-c4-perplexity/perplexity25",
    "lm1-4b2-84b-c4-perplexity/perplexity50",
    "lm1-4b2-84b-c4-perplexity/perplexity2575",


    "lm1-2b8-55b-oscar-repetitions/2b855b55boscar",
    "lm1-2b8-55b-oscarseeds/2b855b55boscarseed1",
    "lm1-2b8-55b-oscarseeds/2b855b55boscarseed2",
    "lm1-2b8-55b-oscarseeds/2b855b55boscarseed3",
    "lm1-2b8-55b-oscarseeds/2b855b55boscarseed4",
    "lm1-2b8-55b-oscar-perplexity/perplexity25",

    "lm1-4b2-84b-oscar-repetitions/4b284b84boscar",
    "lm1-4b2-84b-oscarseeds/4b284b84boscarseed1",
    "lm1-4b2-84b-oscarseeds/4b284b84boscarseed2",
    "lm1-4b2-84b-oscarseeds/4b284b84boscarseed3",
    "lm1-4b2-84b-oscarseeds/4b284b84boscarseed4",

    "lm1-4b2-84b-oscar-perplexity/4b284boscarperplexity25",
]


MODEL_TO_FEWSHOT_SCORES = {}
MODEL_TO_FEWSHOT_SCORES_ACC = {}
MODEL_TO_FEWSHOT_SCORES_GEN = {}

SHOTS = list(range(6))

for MODEL in MODELS:
    MODEL_TO_FEWSHOT_SCORES.setdefault(MODEL, {})
    MODEL_TO_FEWSHOT_SCORES_ACC.setdefault(MODEL, {})
    MODEL_TO_FEWSHOT_SCORES_GEN.setdefault(MODEL, {})

    path = f"{MODEL}/evaluation/generation/merged.csv"
    if not os.path.exists(path):
        print("Skipping: ", {path})
        continue
    generation = pd.read_csv(path)
    for SHOT in SHOTS: 
          
        MODEL_TO_FEWSHOT_SCORES[MODEL].setdefault(SHOT, {})
        MODEL_TO_FEWSHOT_SCORES_ACC[MODEL].setdefault(SHOT, {})
        MODEL_TO_FEWSHOT_SCORES_GEN[MODEL].setdefault(SHOT, {})

        rankeval_files = glob.glob(f"{MODEL}/evaluation/rankeval/*_{SHOT}.csv")
        assert len(rankeval_files) == 1, f"{rankeval_files}"
        rankeval = pd.read_csv(rankeval_files[0])

        # Rescale to 0 - 1, where 0 is random performance
        rankeval["normalized"] = rankeval.apply(lambda x: (x["value"] - TASK_TO_BASELINE[x["task"]]) / (1 - TASK_TO_BASELINE[x["task"]]), axis=1)
        rankeval = rankeval[rankeval["metric"] == "acc"]
        rankeval_scores = rankeval.normalized.values.tolist()

        gen_sub = generation[generation["fewshots"] == SHOT]
        gen_sub = gen_sub[gen_sub["prompt"] != "median"]
        gen_sub = gen_sub[gen_sub["prompt"] != "average"]

        # 0 is already random performance, i.e. no rescaling necessary
        generation_scores = gen_sub.value.values.tolist()
        scores = rankeval_scores + generation_scores
        average_score = sum(scores) / len(scores)

        for i, row in rankeval.iterrows():
            MODEL_TO_FEWSHOT_SCORES[MODEL][SHOT][row["task"]] = row["normalized"]
        for i, row in gen_sub.iterrows():
            MODEL_TO_FEWSHOT_SCORES[MODEL][SHOT][row["dataset"]] = row["value"]

        babi_files = glob.glob(f"{MODEL}/evaluation/*{SHOT}_babi.json")
        if len(babi_files) == 1:
            import json
            with open(babi_files[0], "r") as f:
                MODEL_TO_FEWSHOT_SCORES[MODEL][SHOT]["babi"] = json.load(f)["results"]["babi"]["em"]
        else:
            print(f"Missing bAbI: {MODEL}; Setting to 0 (which is OK for C4)")
            MODEL_TO_FEWSHOT_SCORES[MODEL][SHOT]["babi"] = 0

        #MODEL_TO_FEWSHOT_SCORES[BASE_MODEL][MODEL][SHOT]
        #MODEL_TO_FEWSHOT_SCORES_ACC[BASE_MODEL][MODEL].append(sum(rankeval_scores) / len(rankeval_scores))
        #MODEL_TO_FEWSHOT_SCORES_GEN[BASE_MODEL][MODEL].append(sum(generation_scores) / len(generation_scores))


fatal: destination path 'lm1-2b8-55b-c4-perplexity' already exists and is not an empty directory.
fatal: destination path 'lm1-2b8-55b-c4-repetitions' already exists and is not an empty directory.
fatal: destination path 'lm1-2b8-55b-c4seeds' already exists and is not an empty directory.
fatal: destination path 'lm1-4b2-84b-c4-perplexity' already exists and is not an empty directory.
fatal: destination path 'lm1-4b2-84b-c4-repetitions' already exists and is not an empty directory.
fatal: destination path 'lm1-4b2-84b-c4seeds' already exists and is not an empty directory.
fatal: destination path 'lm1-2b8-55b-oscar-perplexity' already exists and is not an empty directory.
fatal: destination path 'lm1-2b8-55b-oscar-repetitions' already exists and is not an empty directory.
fatal: destination path 'lm1-2b8-55b-oscarseeds' already exists and is not an empty directory.
fatal: destination path 'lm1-4b2-84b-oscar-perplexity' already exists and is not an empty directory.
fatal: destination path

In [6]:
### Latex Table ###

import numpy as np

TASK_TO_NAME = {
    "anli_r1": "ANLI R1",
    "anli_r2": "ANLI R2",
    "anli_r3": "ANLI R3",
    "arc_challenge": "ARC-Challenge",
    "arc_easy": "ARC-Easy",
    "boolq": "BoolQ",
    "cb": "CB",
    "copa": "COPA",
    "hellaswag": "HellaSwag",
    "piqa": "PiQA",
    "rte": "RTE",
    "sciq": "SciQ",
    "storycloze_2016": "StoryCloze 2016",
    "winogrande": "WinoGrande XL",

    "e2e_nlg_cleaned": "E2E NLG",
    "gem_xsum": "XSUM",
    "web_nlg_en": "WebNLG EN",
    "wiki_lingua_en": "WikiLingua EN",

    "babi": "bAbI",    
}

### Table 1: OSCAR + C4 2b8
average = True
SHOT = 5

TABLE = """
\\begin{table}
  \\centering
  \\caption{Perplexity}
  \\resizebox{\\textwidth}{!}{
    \\begin{tabular}{l|lllllll|llll}
"""

HEADER_A = "Training Data & \multicolumn{7}{c|}{C4} & \multicolumn{4}{c}{OSCAR}" + " \\\\"
HEADER_B = "Parameters & \multicolumn{3}{c|}{2.8B parameters} & \multicolumn{4}{c|}{4.2B parameters} & \multicolumn{2}{c|}{2.8B parameters} & \multicolumn{2}{c|}{4.2B parameters}" + " \\\\"
#HEADER_C = "Dataset & No filtering & Top 25\\% perplexity & Top 50\\% Perplexity & Top 25\\% perplexity & Top 50\\% Perplexity & 25\\% - 75\\% perplexity & No filtering & Top 25\\% perplexity & No filtering & Top 25\\% perplexity" + " \\\\"
HEADER_C = "Dataset & All & Top 25\\% & Top 50\\% & All & 25\\% & 50\\% & 25\\% - 75\\% & All & Top 25\\% & All & Top 25\\% perplexity" + " \\\\"

TABLE += "\n\\midrule\n".join([HEADER_A, HEADER_B, HEADER_C])

for task in TASK_TO_NAME:
    scores_str = []
    for model in MODELS:
        if "seed" in model: continue
        if "perplexity" not in model:
            model_scores = []
            modelseeds = model.replace("-repetitions", "seeds")
            for model_name in [model+"", modelseeds+"seed1", modelseeds+"seed2", modelseeds+"seed3", modelseeds+"seed4"]:
                model_scores.append(sum([MODEL_TO_FEWSHOT_SCORES[model_name][SHOT][task] for SHOT in [0, 1, 2, 3, 4, 5]]) / 6)
            avg = np.mean(model_scores)
            std = np.std(model_scores)
            scores_str.append(f"{round(avg * 100, 1)} ± {str(round(std * 100, 1))}")
            
        else:
            avg = sum([MODEL_TO_FEWSHOT_SCORES[model][SHOT][task] for SHOT in [0, 1, 2, 3, 4, 5]]) / 6
            scores_str.append(str(round(avg * 100, 1)))
    
    TABLE += "\n" + f"{TASK_TO_NAME[task]} & " + " & ".join(scores_str) + " \\\\"

    if task in ("winogrande", "wiki_lingua_en", "babi"):
        TABLE += "\n" + "\\midrule"


# Add average
scores_avg_str = []
for model in MODELS:
    if "seed" in model: continue
    if "perplexity" not in model:
        model_scores = []
        modelseeds = model.replace("-repetitions", "seeds")
        for model_name in [model+"", modelseeds+"seed1", modelseeds+"seed2", modelseeds+"seed3", modelseeds+"seed4"]:
            scores = [list(MODEL_TO_FEWSHOT_SCORES[model_name][SHOT].values()) for SHOT in [0, 1, 2, 3, 4, 5]]
            for task_scores in scores:
                assert len(task_scores) == len(TASK_TO_NAME)            
            model_scores.append(
                np.mean([sub_score for shot_scores in scores for sub_score in shot_scores])
            )
        avg = np.mean(model_scores)
        std = np.std(model_scores)
        scores_avg_str.append(f"{round(avg * 100, 1)} ± {str(round(std * 100, 1))}")

    else:
        scores = [list(MODEL_TO_FEWSHOT_SCORES[model][SHOT].values()) for SHOT in [0, 1, 2, 3, 4, 5]]
        for task_scores in scores:
            assert len(task_scores) == len(TASK_TO_NAME)
        score_avg = [sub_score for shot_scores in scores for sub_score in shot_scores]
        avg = sum(score_avg) / len(score_avg)
        scores_avg_str.append(f"{round(avg * 100, 1)}")

#std = np.std(scores_avg[:5])
#scores_avg_std = str(round(((sum(scores_avg[:5]) / 5) * 100), 1)) + f" ± {str(round(std * 100, 1))}"

#std_oscar = np.std(scores_avg[-6:-1])
#scores_avg_std_oscar = str(round(((sum(scores_avg[-6:-1]) / 5) * 100), 1)) + f" ± {str(round(std_oscar * 100, 1))}"

#scores_avg_str_c4 = scores_avg_std + " & " + " & ".join([str(round(score_avg * 100, 1)) for score_avg in scores_avg[5:-6]])
#scores_avg_str_oscar = scores_avg_std_oscar + " & " + " & ".join([str(round(score_avg * 100, 1)) for score_avg in scores_avg[-1:]])


#TABLE += "\nAverage & " + scores_avg_str_c4 + " & " + scores_avg_str_oscar + " \\\\"

TABLE += "\nAverage & " + " & ".join(scores_avg_str) + " \\\\"
TABLE += "\n\\bottomrule"

TABLE += """
\bottomrule
  \end{tabular}
  }
  \label{tab:pplx}
\end{table}
"""

print(TABLE)


\begin{table}
  \centering
  \caption{Perplexity}
  \resizebox{\textwidth}{!}{
    \begin{tabular}{l|lllllll|llll}
Training Data & \multicolumn{7}{c|}{C4} & \multicolumn{4}{c}{OSCAR} \\
\midrule
Parameters & \multicolumn{3}{c|}{2.8B parameters} & \multicolumn{4}{c|}{4.2B parameters} & \multicolumn{2}{c|}{2.8B parameters} & \multicolumn{2}{c|}{4.2B parameters} \\
\midrule
Dataset & All & Top 25\% & Top 50\% & All & 25\% & 50\% & 25\% - 75\% & All & Top 25\% & All & Top 25\% perplexity \\
ANLI R1 & 0.4 ± 1.6 & -0.1 & 0.9 & -0.5 ± 1.4 & -0.0 & -0.7 & -0.8 & -0.3 ± 0.5 & -0.4 & -0.4 ± 1.2 & -2.2 \\
ANLI R2 & 0.9 ± 0.4 & -0.2 & -0.7 & 0.0 ± 1.3 & -0.4 & -0.0 & 1.1 & 1.0 ± 1.0 & 1.7 & 1.0 ± 0.9 & 0.7 \\
ANLI R3 & 1.7 ± 0.5 & 0.5 & 1.4 & 0.7 ± 0.5 & 0.7 & 2.9 & 0.4 & 0.4 ± 0.8 & 1.7 & 1.2 ± 0.5 & 2.1 \\
ARC-Challenge & 1.6 ± 1.0 & 3.3 & 2.9 & 4.2 ± 1.6 & 10.2 & 9.3 & 7.9 & -1.4 ± 0.8 & 3.3 & 1.8 ± 0.8 & 6.3 \\
ARC-Easy & 44.5 ± 0.5 & 47.3 & 47.7 & 48.1 ± 4.8 & 55.8 & 53.7 & 51.0 & 39.7 ± 0.3

In [5]:
import glob
import os
import pandas as pd

!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-2b8-55b-dedup
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-2b8-55b-c4-repetitions
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-2b8-55b-c4seeds


!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-4b2-84b-c4-dedup
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-4b2-84b-c4-repetitions
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-4b2-84b-c4seeds

!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-2b8-55b-oscardedup25
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-2b8-55b-oscar-repetitions
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-2b8-55b-oscarseeds

!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-4b2-84b-oscar-dedup
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-4b2-84b-oscar-repetitions
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-4b2-84b-oscarseeds

!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-2b8-55b-oscarroots
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datablations/lm1-4b2-84b-oscarroots


TASK_TO_BASELINE = {
    "anli_r1": 1/3,
    "anli_r2": 1/3,
    "anli_r3": 1/3,
    "arc_challenge": 1/4,
    "arc_easy": 1/4,
    "boolq": 1/2,
    "cb": 1/3,
    "copa": 1/2,
    "hellaswag": 1/4,
    "piqa": 1/2,
    "rte": 1/2,
    "sciq": 1/4,
    "storycloze_2016": 1/4,
    "winogrande": 1/2,
    "babi": 0,
}

MODELS = [
    "lm1-2b8-55b-c4-repetitions/2b855b55bc4",
    "lm1-2b8-55b-c4seeds/2b855b55bc4seed1",
    "lm1-2b8-55b-c4seeds/2b855b55bc4seed2",
    "lm1-2b8-55b-c4seeds/2b855b55bc4seed3",
    "lm1-2b8-55b-c4seeds/2b855b55bc4seed4",
    "lm1-2b8-55b-dedup",

    "lm1-4b2-84b-c4-repetitions/4b284b84bc4",
    "lm1-4b2-84b-c4seeds/4b284b84bc4seed1",
    "lm1-4b2-84b-c4seeds/4b284b84bc4seed2",
    "lm1-4b2-84b-c4seeds/4b284b84bc4seed3",
    "lm1-4b2-84b-c4seeds/4b284b84bc4seed4",
    "lm1-4b2-84b-c4-dedup/4b284bc4dedup",

    "lm1-2b8-55b-oscar-repetitions/2b855b55boscar",
    "lm1-2b8-55b-oscarseeds/2b855b55boscarseed1",
    "lm1-2b8-55b-oscarseeds/2b855b55boscarseed2",
    "lm1-2b8-55b-oscarseeds/2b855b55boscarseed3",
    "lm1-2b8-55b-oscarseeds/2b855b55boscarseed4",
    "lm1-2b8-55b-oscardedup25/",
    "lm1-2b8-55b-oscarroots",

    "lm1-4b2-84b-oscar-repetitions/4b284b84boscar",
    "lm1-4b2-84b-oscarseeds/4b284b84boscarseed1",
    "lm1-4b2-84b-oscarseeds/4b284b84boscarseed2",
    "lm1-4b2-84b-oscarseeds/4b284b84boscarseed3",
    "lm1-4b2-84b-oscarseeds/4b284b84boscarseed4",
    "lm1-4b2-84b-oscar-dedup/4b284b84boscardedup25expanded",
    "lm1-4b2-84b-oscarroots",
]


MODEL_TO_FEWSHOT_SCORES = {}
MODEL_TO_FEWSHOT_SCORES_ACC = {}
MODEL_TO_FEWSHOT_SCORES_GEN = {}

SHOTS = list(range(6))

for MODEL in MODELS:
    MODEL_TO_FEWSHOT_SCORES.setdefault(MODEL, {})
    MODEL_TO_FEWSHOT_SCORES_ACC.setdefault(MODEL, {})
    MODEL_TO_FEWSHOT_SCORES_GEN.setdefault(MODEL, {})

    path = f"{MODEL}/evaluation/generation/merged.csv"
    if not os.path.exists(path):
        print("Skipping: ", {path})
        continue
    generation = pd.read_csv(path)
    for SHOT in SHOTS: 
          
        MODEL_TO_FEWSHOT_SCORES[MODEL].setdefault(SHOT, {})
        MODEL_TO_FEWSHOT_SCORES_ACC[MODEL].setdefault(SHOT, {})
        MODEL_TO_FEWSHOT_SCORES_GEN[MODEL].setdefault(SHOT, {})

        rankeval_files = glob.glob(f"{MODEL}/evaluation/rankeval/*_{SHOT}.csv")
        assert len(rankeval_files) == 1, f"{rankeval_files}"
        rankeval = pd.read_csv(rankeval_files[0])

        # Rescale to 0 - 1, where 0 is random performance
        rankeval["normalized"] = rankeval.apply(lambda x: (x["value"] - TASK_TO_BASELINE[x["task"]]) / (1 - TASK_TO_BASELINE[x["task"]]), axis=1)
        rankeval = rankeval[rankeval["metric"] == "acc"]
        rankeval_scores = rankeval.normalized.values.tolist()

        gen_sub = generation[generation["fewshots"] == SHOT]
        gen_sub = gen_sub[gen_sub["prompt"] != "median"]
        gen_sub = gen_sub[gen_sub["prompt"] != "average"]

        # 0 is already random performance, i.e. no rescaling necessary
        generation_scores = gen_sub.value.values.tolist()
        scores = rankeval_scores + generation_scores
        average_score = sum(scores) / len(scores)

        for i, row in rankeval.iterrows():
            MODEL_TO_FEWSHOT_SCORES[MODEL][SHOT][row["task"]] = row["normalized"]
        for i, row in gen_sub.iterrows():
            MODEL_TO_FEWSHOT_SCORES[MODEL][SHOT][row["dataset"]] = row["value"]

        babi_files = glob.glob(f"{MODEL}/evaluation/*{SHOT}_babi.json")
        if len(babi_files) == 1:
            import json
            with open(babi_files[0], "r") as f:
                MODEL_TO_FEWSHOT_SCORES[MODEL][SHOT]["babi"] = json.load(f)["results"]["babi"]["em"]
        else:
            print(f"Missing bAbI: {MODEL}; Setting to 0 (which is OK for C4)")
            MODEL_TO_FEWSHOT_SCORES[MODEL][SHOT]["babi"] = 0

        #MODEL_TO_FEWSHOT_SCORES[BASE_MODEL][MODEL][SHOT]
        #MODEL_TO_FEWSHOT_SCORES_ACC[BASE_MODEL][MODEL].append(sum(rankeval_scores) / len(rankeval_scores))
        #MODEL_TO_FEWSHOT_SCORES_GEN[BASE_MODEL][MODEL].append(sum(generation_scores) / len(generation_scores))


fatal: destination path 'lm1-2b8-55b-dedup' already exists and is not an empty directory.
fatal: destination path 'lm1-2b8-55b-c4-repetitions' already exists and is not an empty directory.
fatal: destination path 'lm1-2b8-55b-c4seeds' already exists and is not an empty directory.
fatal: destination path 'lm1-4b2-84b-c4-dedup' already exists and is not an empty directory.
fatal: destination path 'lm1-4b2-84b-c4-repetitions' already exists and is not an empty directory.
fatal: destination path 'lm1-4b2-84b-c4seeds' already exists and is not an empty directory.
fatal: destination path 'lm1-2b8-55b-oscardedup25' already exists and is not an empty directory.
fatal: destination path 'lm1-2b8-55b-oscar-repetitions' already exists and is not an empty directory.
fatal: destination path 'lm1-2b8-55b-oscarseeds' already exists and is not an empty directory.
fatal: destination path 'lm1-4b2-84b-oscar-dedup' already exists and is not an empty directory.
fatal: destination path 'lm1-4b2-84b-oscar-re

In [6]:
### Latex Table ###

import numpy as np

TASK_TO_NAME = {
    "anli_r1": "ANLI R1",
    "anli_r2": "ANLI R2",
    "anli_r3": "ANLI R3",
    "arc_challenge": "ARC-Challenge",
    "arc_easy": "ARC-Easy",
    "boolq": "BoolQ",
    "cb": "CB",
    "copa": "COPA",
    "hellaswag": "HellaSwag",
    "piqa": "PiQA",
    "rte": "RTE",
    "sciq": "SciQ",
    "storycloze_2016": "StoryCloze 2016",
    "winogrande": "WinoGrande XL",

    "e2e_nlg_cleaned": "E2E NLG",
    "gem_xsum": "XSUM",
    "web_nlg_en": "WebNLG EN",
    "wiki_lingua_en": "WikiLingua EN",

    "babi": "bAbI",  
}

### Table 1: OSCAR + C4 2b8
average = True
SHOT = 5

TABLE = """
\\begin{table}
  \\centering
  \\caption{\textbf{Results after filtering with deduplication.}}
  \\resizebox{\\textwidth}{!}{
    \\begin{tabular}{l|llll|llll}
    \toprule    
"""

HEADER_A = "Training Data & \multicolumn{4}{c|}{C4} & \multicolumn{4}{c}{OSCAR}" + " \\\\"
HEADER_B = "Parameters & \multicolumn{2}{c|}{2.8B parameters} & \multicolumn{2}{c|}{4.2B parameters} & \multicolumn{2}{c|}{2.8B parameters} & \multicolumn{2}{c|}{4.2B parameters}" + " \\\\"
HEADER_C = "Dataset & All & Dedup & All & Dedup & All & Dedup & All & Dedup" + " \\\\"

TABLE += "\n\\midrule\n".join([HEADER_A, HEADER_B, HEADER_C])

for task in TASK_TO_NAME:
    scores_str = []
    for model in MODELS:
        if "seed" in model: continue
        if ("dedup" not in model) and ("roots" not in model):
            model_scores = []
            modelseeds = model.replace("-repetitions", "seeds")
            for model_name in [model+"", modelseeds+"seed1", modelseeds+"seed2", modelseeds+"seed3", modelseeds+"seed4"]:
                model_scores.append(sum([MODEL_TO_FEWSHOT_SCORES[model_name][SHOT][task] for SHOT in [0, 1, 2, 3, 4, 5]]) / 6)
            avg = np.mean(model_scores)
            std = np.std(model_scores)
            scores_str.append(f"{round(avg * 100, 1)} ± {str(round(std * 100, 1))}")
            
        else:
            avg = sum([MODEL_TO_FEWSHOT_SCORES[model][SHOT][task] for SHOT in [0, 1, 2, 3, 4, 5]]) / 6
            scores_str.append(str(round(avg * 100, 1)))
    
    TABLE += "\n" + f"{TASK_TO_NAME[task]} & " + " & ".join(scores_str) + " \\\\"

    if task in ("winogrande", "wiki_lingua_en", "babi"):
        TABLE += "\n" + "\\midrule"


# Add average
scores_avg_str = []
for model in MODELS:
    if "seed" in model: continue
    if ("dedup" not in model) and ("roots" not in model):
        model_scores = []
        modelseeds = model.replace("-repetitions", "seeds")
        for model_name in [model+"", modelseeds+"seed1", modelseeds+"seed2", modelseeds+"seed3", modelseeds+"seed4"]:
            scores = [list(MODEL_TO_FEWSHOT_SCORES[model_name][SHOT].values()) for SHOT in [0, 1, 2, 3, 4, 5]]
            for task_scores in scores:
                assert len(task_scores) == len(TASK_TO_NAME)            
            model_scores.append(
                np.mean([sub_score for shot_scores in scores for sub_score in shot_scores])
            )
        avg = np.mean(model_scores)
        std = np.std(model_scores)
        scores_avg_str.append(f"{round(avg * 100, 1)} ± {str(round(std * 100, 1))}")

    else:
        scores = [list(MODEL_TO_FEWSHOT_SCORES[model][SHOT].values()) for SHOT in [0, 1, 2, 3, 4, 5]]
        for task_scores in scores:
            assert len(task_scores) == len(TASK_TO_NAME)
        score_avg = [sub_score for shot_scores in scores for sub_score in shot_scores]
        avg = sum(score_avg) / len(score_avg)
        scores_avg_str.append(f"{round(avg * 100, 1)}")

TABLE += "\nAverage & " + " & ".join(scores_avg_str) + " \\\\"
TABLE += "\n\\bottomrule"

TABLE += """
\bottomrule
  \end{tabular}
  }
  \label{tab:dedup}
\end{table}
"""

print(TABLE)


\begin{table}
  \centering
  \caption{	extbf{Results after filtering with deduplication.}}
  \resizebox{\textwidth}{!}{
    \begin{tabular}{l|llll|llll}
    	oprule    
Training Data & \multicolumn{4}{c|}{C4} & \multicolumn{4}{c}{OSCAR} \\
\midrule
Parameters & \multicolumn{2}{c|}{2.8B parameters} & \multicolumn{2}{c|}{4.2B parameters} & \multicolumn{2}{c|}{2.8B parameters} & \multicolumn{2}{c|}{4.2B parameters} \\
\midrule
Dataset & All & Dedup & All & Dedup & All & Dedup & All & Dedup \\
ANLI R1 & 0.4 ± 1.6 & -0.2 & -0.5 ± 1.4 & -0.8 & -0.3 ± 0.5 & -2.1 & -1.7 & -0.4 ± 1.2 & -1.8 & 1.2 \\
ANLI R2 & 0.9 ± 0.4 & 1.1 & 0.0 ± 1.3 & -0.1 & 1.0 ± 1.0 & 2.0 & 0.7 & 1.0 ± 0.9 & -0.5 & -0.3 \\
ANLI R3 & 1.7 ± 0.5 & 1.8 & 0.7 ± 0.5 & 0.4 & 0.4 ± 0.8 & 0.4 & 0.2 & 1.2 ± 0.5 & 0.8 & -0.3 \\
ARC-Challenge & 1.6 ± 1.0 & 0.6 & 4.2 ± 1.6 & 3.9 & -1.4 ± 0.8 & 2.6 & -0.9 & 1.8 ± 0.8 & 6.8 & 0.6 \\
ARC-Easy & 44.5 ± 0.5 & 43.0 & 48.1 ± 4.8 & 46.8 & 39.7 ± 0.3 & 44.6 & 42.3 & 45.7 ± 0.6 & 51.0 & 47.1 \